In [ ]:
import caffe
import numpy as np

In [ ]:
ICIPpath = '/home/jogue/workspace/ICIPmodels/'
mono_deploy = ICIPpath + 'mono_deploy.prototxt'
fuse_deploy = ICIPpath + 'fuse_deploy.prototxt'
weights_path_0='/home/jogue/workspace/ICIPmodels/rgb-10-10-100/weights.caffemodel'
weights_path_1='/home/jogue/workspace/ICIPmodels/DHA/weights.caffemodel'
#weights_path_1='/home/jguerry/workspace/ICIPmodels/cube_weights.caffemodel'
weights_path_fuse='/home/jogue/workspace/ICIPmodels/fuse_weights.caffemodel'

In [ ]:
def transfer(new_net, net, suffix=''):
    # from fcn.berkeleyvision.org
    for p in net.params:
        #print "Working on", p, ':'
        p_new = p + suffix
        if p_new not in new_net.params:
            print 'dropping', p
            continue
        for i in range(len(net.params[p])):
            if i > (len(new_net.params[p_new]) - 1):
                print 'dropping', p, i
                break
            if net.params[p][i].data.shape != new_net.params[p_new][i].data.shape:
                print 'coercing', p, i, 'from', net.params[p][i].data.shape, 'to', new_net.params[p_new][i].data.shape
            else:
                print 'copying', p, ' -> ', p_new, i
            new_net.params[p_new][i].data.flat = net.params[p][i].data.flat

In [ ]:
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [ ]:
monoNet = caffe.Net(mono_deploy, caffe.TEST)
monoBlobs = [(k, v.data.shape) for k, v in monoNet.blobs.items()]
monoLayers = [(k, v[0].data.shape, v[1].data.shape) for k, v in monoNet.params.items() if k != 'upscore37']

In [ ]:
fuseNet = caffe.Net(fuse_deploy, caffe.TEST)
fuseBlobs = [(k, v.data.shape) for k, v in fuseNet.blobs.items()]
fuseLayers = [(k, v[0].data.shape, v[1].data.shape) for k, v in fuseNet.params.items() if k not in ['upscore37_data0','upscore37_data1','upscore_sum', 'upscore_concat','upscore_rc_corrected']]

In [ ]:
monoNet.copy_from(weights_path_0,)
transfer(fuseNet,monoNet,'_data0')
monoNet.copy_from(weights_path_1)
transfer(fuseNet,monoNet,'_data1')

In [ ]:
for upscore_layer in ['upscore37_data0','upscore37_data1','upscore_sum', 'upscore_concat','upscore_rc_corrected']:
    for i in range(len(fuseNet.params[upscore_layer])):
        fuseNet.params[upscore_layer][i].data.flat = monoNet.params['upscore37'][i].data.flat
        print(upscore_layer)
        print(np.sum(fuseNet.params[upscore_layer][i].data)) #flat = net.params[p][i].data.flat

In [ ]:
#### SAVING
print "Saving output model to %s " % weights_path_fuse
fuseNet.save(weights_path_fuse)

In [ ]:
del fuseNet
del monoNet